## Classificação de filtros

Uma categoria significativa de sistemas LIT abrange aqueles em que a resposta em frequência é igual a um em uma faixa específica de frequências e é igual a zero nas outras frequências. Isso se refere aos filtros ideais de seleção de frequência e dentre eles, é possível citar 4 importantes categorias:

- **Passa baixa** (PB ou LP) - Também conhecido como Lowpass em inglês, estes filtros selecionam frequências baixas até um um valor pré-definido $f_c$ e atenuam frequências acima de $fc$.
- **Passa alta** (PA ou HP) - Também conhecido como Highpass em inglês, estes filtros selecionam frequências acima de um valor pré-definido $f_c$ e atenuam frequências abaixo desse mesmo fc.
- **Passa-faixa** (PF ou BP) - Também conhecido como Bandpass em inglês, estes filtros selecionam frequências em um intervalo pré-definido $f_{cl}$ e $f_{ch}$, atenuando frequências fora dessas faixas.
- **Rejeita-faixa** (RF ou BR) - Também conhecido como Bandreject ou Bandstop em inglês, estes filtros rejeitam (atenuam) frequências em um intervalo pré-definido $f_{cl}$ e $f_{ch}$, selecionando apenas as frequências abaixo de $f_{cl}$ e acima de $f_{ch}$.

Abaixo, vamos definir uma função que irá aplicar todos os filtros definidos nessa seção. 

Importaremos todas as funções apresentadas no capítulo de introdução mais as funções `butter` e `lfilter` da função SciPy.

In [ ]:
%run -i 1introducao.ipynb

from scipy.signal import butter, lfilter

Importante notar o uso do teorema de Nyquist, que estabelece que a taxa de amostragem deve ser pelo menos duas vezes a frequência mais alta esperada no sinal. Isso é feita para para evitar perdas de informação na conversão analógica para digital ou digital para analógica
Nessa função $[b, a]$ são o numerador e o denominador respectivamente da função de transferência do filtro. Com essa informação usamos a função $lfilter$ que utiliza uma estrutura em __forma direta II transposta__ para aplicar o filtro. O filtro usado é o filtro Butterworth que por padrão utiliza o __modo canônico__.

In [ ]:
def run_filter(filter_chose_type, cutoff_frequency_int, file_name, order, cutoff_frequency_high = -1):
    # leitura do arquivo WAV
    sample_rate, input_signal = read_audio_file(file_name)

    # aplicação do filtro
    filtered_data, w, h = butter_filter(input_signal, cutoff_frequency_int, sample_rate, filter_chose_type, order, cutoff_frequency_high)

    # plotagem da resposta em frequência do filtro
    %matplotlib inline
    plot_filter(cutoff_frequency_int, sample_rate, w, h)

    # escrita do áudio de saída em arquivo
    write_audio_file(filtered_data, sample_rate, 'output_file_classificacao.wav')

    # plotagem dos sinais de entrada e saída no domínio do tempo e da frequência
    %matplotlib ipympl
    plot_signal(input_signal, sample_rate, 'Sinal de Entrada')
    plot_signal(filtered_data, sample_rate, 'Sinal de Saída')

    # mostra os reprodutores de áudio do sinal de entrada e de saída
    display_audio(file_name, 'output_file_classificacao.wav')


In [ ]:
# função para aplicar o filtro canônico no sinal
def butter_filter(data, cutoff_frequency, sample_rate, filter_class, order, cutoff_frequency_high):
    # aplicação do teorema de nyquist e normalização das frequências de corte
    nyq = 0.5 * sample_rate
    if cutoff_frequency_high != -1:
        normal_cutoff = []
        normal_cutoff.append(cutoff_frequency/nyq)
        normal_cutoff.append(cutoff_frequency_high/nyq)
    else:
        normal_cutoff = cutoff_frequency / nyq

    
    # aplicação do filtro Butterworth
    b, a = butter(order, normal_cutoff, btype=filter_class, analog=False)
    
    # calcula a resposta em frequência do filtro.
    w, h = freqz(b, a, fs=sample_rate)

    # aplica o filtro com estrutura direta II transposta
    filtered_data = lfilter(b, a, data) 
    return filtered_data, w, h

# reseta os widgets
clear_output()
out1.clear_output()
out2.clear_output()
out3.clear_output()

# roda o filtro e apresenta o widget
run_filter('highpass', 3000, 'piano.wav', 5)
widgets_basic_filters()
